In [15]:
from datetime import datetime
from langchain_core.tools import tool
import json
from langchain_groq import ChatGroq
import requests
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
sectors_api_key = os.getenv("SECTORS_API_KEY")

llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key = groq_api_key)

@tool
def get_daily_transaction_data(stock: str, start_date:str, end_date:str ) -> str:
    """Get daily transaction data for stock, such as close price, volume, and market cap on certain date"""

    url = f"https://api.sectors.app/v1/daily/{stock}/?start={start_date}&end={end_date}"
    api_key = sectors_api_key

    headers = {"Authorization": api_key}
    response = requests.get(url, headers = headers)

    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        print(response.status_code)
    return result

@tool
def get_top_companies_based_on_transaction_volume(start_date:str, end_date:str, top_n:str) -> str:
    """Get top companies based on transaction volume"""

    url = f"https://api.sectors.app/v1/most-traded/?start={start_date}&end={end_date}&n_stock={top_n}"
    api_key = sectors_api_key

    headers = {"Authorization": api_key}
    response = requests.get(url, headers = headers)

    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        print(response.status_code)
    return result

@tool
def get_company_info(stock:str) -> str:
  """ Get company info, such as company name, listing board, industry, sub industry, sector, sub sector, market cap, market cap rank, address, employee num, listing date, website, phone, email, last close price, latest close date, daily close change"""

  url = f"https://api.sectors.app/v1/company/report/{stock}/?sections=overview"

  headers = {"Authorization": sectors_api_key}
  response = requests.get(url, headers = headers)

  if response.status_code == 200:
    data = response.json()
    result = json.dumps(data)
  else:
    print(response.status_code)
  return result


# SIMPLE QUERY
simple_query = "Give me the top 3 companies based on transaction volume on 2 April 2024"

# INTERMEDIATE QUERY
intermediate_query = "Based on the closing prices of BBRI from April 1 to April 28, is the trend positive or negative? Give me the detailed reason"

# COMPLEX QUERY
complex_query = "What were the market cap of BBCA and BREN? Which one was higher? For a stock with a higher market capitalization, could you also provide me with the email, phone number, and website, as I am interested in investing there?"

queries = [simple_query, intermediate_query, complex_query]

tools = [get_daily_transaction_data, get_top_companies_based_on_transaction_volume, get_company_info]

llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key = groq_api_key)
llm_with_tools = llm.bind_tools(tools)


In [18]:
def generate_output(query):
    messages = [("system", "You're a helpful financial assistant"), ("human", query)]
    ai_msg = llm_with_tools.invoke(messages)
    # print(ai_msg.tool_calls)
    for tool_call in ai_msg.tool_calls:
        selected_tool = {"get_daily_transaction_data": get_daily_transaction_data, 
                        "get_top_companies_based_on_transaction_volume": get_top_companies_based_on_transaction_volume,
                        "get_company_info":get_company_info}[tool_call["name"].lower()]
        tool_output = selected_tool.invoke(tool_call["args"])
    return tool_output

In [19]:
generate_output(simple_query)

'{"2024-04-02": [{"symbol": "GOTO.JK", "volume": 2224757900, "price": 70}, {"symbol": "DOOH.JK", "volume": 675247000, "price": 57}, {"symbol": "PNLF.JK", "volume": 502243100, "price": 310}]}'

In [20]:
generate_output(intermediate_query)

'[{"symbol": "BBRI.JK", "date": "2022-04-01", "close": 4730, "volume": 145105000, "market_cap": null}, {"symbol": "BBRI.JK", "date": "2022-04-04", "close": 4700, "volume": 103902000, "market_cap": null}, {"symbol": "BBRI.JK", "date": "2022-04-05", "close": 4700, "volume": 76154300, "market_cap": null}, {"symbol": "BBRI.JK", "date": "2022-04-06", "close": 4600, "volume": 111009800, "market_cap": null}, {"symbol": "BBRI.JK", "date": "2022-04-07", "close": 4630, "volume": 91942500, "market_cap": null}, {"symbol": "BBRI.JK", "date": "2022-04-08", "close": 4610, "volume": 125657900, "market_cap": null}, {"symbol": "BBRI.JK", "date": "2022-04-11", "close": 4570, "volume": 109861100, "market_cap": null}, {"symbol": "BBRI.JK", "date": "2022-04-12", "close": 4610, "volume": 96961200, "market_cap": null}, {"symbol": "BBRI.JK", "date": "2022-04-13", "close": 4610, "volume": 112327600, "market_cap": null}, {"symbol": "BBRI.JK", "date": "2022-04-14", "close": 4540, "volume": 150659600, "market_cap"

In [22]:
generate_output(complex_query)

'{"symbol": "BREN.JK", "company_name": "PT Barito Renewables Energy Tbk.", "overview": {"listing_board": "Main", "industry": "Electric Utilities", "sub_industry": "Electric Utilities", "sector": "Infrastructures", "sub_sector": "Utilities", "market_cap": 1257588435976192, "market_cap_rank": 1, "address": "Wisma Barito Pacific II, Lantai 23,\\nJl. Let. Jend. S. Parman Kav. 60, RT 010, RW 005, Slipi, Palmerah, Jakarta 11410,\\nIndonesia", "employee_num": 634, "listing_date": "2023-10-09", "website": "www.baritorenewables.co.id", "phone": "(021) 530 6711", "email": "corpsec@baritorenewables.co.id", "last_close_price": 9400, "latest_close_date": "2024-06-24", "daily_close_change": 0.032967032967033}}'